# Agents

Agents are the core building block in your apps. An agent is a large language model (LLM), configured with instructions and tools.

## Basic configuration

Most common properties of an agent we will configure are:
  - `name` : Required string to identify the agent.
  - `instructions` : `Developer message` || `system prompt`
  - `model` : which LLM to use, and optional model_settings to configure model tuning parameters like temperature, top_p, etc.
  - `tools` : List of tools to use in the agent.
 

In [4]:
!python -m pip install --upgrade pip

In [ ]:
!pip install openai-agents

In [8]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city : str) -> str:
    """
    Get the current weather for a given city.
    """
    return f"Current weather in {city} is sunny."

agent = Agent(
    name="WeatherBot",
    instructions="You are a helpful assistant that provides weather information.",
    model="o3-mini",
    tools=[get_weather],
)


## Context

### 1. **Generic Context Type**
`Agents are generic on their context type` means that the `agent` is designed to work with any kind of `context`, without being tied to a specific type. This allows the agent to be flexible and adaptable to different use cases.

### 2. **Context Type**
`ContextType` is a type that represents the type of context that the agent is working with. It can be any type that the agent needs to interact with, such as a `list`, `dictionary`, `string`, or any other data structure.

- **Context as Dependency Injection**:  
  The context can be seen as a dependency that the agent needs to function. This dependency can be injected into the agent through its constructor or other methods, allowing the agent to be decoupled from any specific implementation details.

### 3. **How it Works**
- When the `Runner.run()` method is executed, a `Context` object is created and passed into the agent.
- This context is passed along to every agent, tool, handoff, or component used during the agent's run.
- The context is used to store and share dependencies and state between different components.

### 4. **Example Usage**

```python
class MyContext:
    def __init__(self, config, database):
        self.config = config
        self.database = database

class MyAgent:
    def __init__(self, context):
        self.context = context

    def run(self):
        print(f"Connecting to database: {self.context.database}")
        print(f"Using config: {self.context.config}")

# Create a context object with the required dependencies
context = MyContext(config={"api_key": "your-api-key"}, database="my_database")

# Pass the context to the agent
agent = MyAgent(context)

# Run the agent
agent.run()


Here is official docs example

```python
@dataclass
class UserContext:
    uid: str
    is_pro_user: bool

    async def fetch_purchases() -> list[Purchase]:
        return ...

agent = Agent[UserContext](
    ...,
)
```

## **Output types**

By default, agents produce plain text (i.e. str) outputs. 
If you want the agent to produce a particular type of output, you can use the `output_type` parameter.
A common choice is to use `Pydantic` `objects`, but we support any type that can be wrapped in a Pydantic `TypeAdapter` - dataclasses, lists, TypedDict, etc.

```python
    from pydantic import BaseModel
    from agents import Agent


    class CalendarEvent(BaseModel):
        name: str
        date: str
        participants: list[str]

    agent = Agent(
        name="Calendar extractor",
        instructions="Extract calendar events from text",
        output_type=CalendarEvent,
    )
```

## **`Note`**

When you pass an `output_type`, that tells the model to use structured outputs instead of regular plain text responses.



### **Handoffs**:

  * Sub-agents that the main agent can delegate tasks to.
  * You provide a list of handoffs, and the agent can delegate to the relevant sub-agent based on the situation.
  * This enables modular, specialized agents that focus on specific tasks, improving efficiency and flexibility.

* **Example Setup**:

  * Create agents that specialize in specific tasks:

    ```python
    from agents import Agent

    booking_agent = Agent(...)
    refund_agent = Agent(...)
    ```

* **Triage Agent with Handoffs**:

  * The triage agent manages queries and delegates to the appropriate agent (e.g., booking or refund) based on the user’s inquiry.

    ```python
    triage_agent = Agent(
        name="Triage agent",
        instructions=(
            "Help the user with their questions. "
            "If they ask about booking, handoff to the booking agent. "
            "If they ask about refunds, handoff to the refund agent."
        ),
        handoffs=[booking_agent, refund_agent],
    )
    ```


### *Dynamic instructions*

We can provide instructions when you create the agent.

However, you can also provide dynamic instructions via a function. The function will receive the agent and context, and must return the prompt. 

Both regular and async functions are accepted.

```python
def dynamic_instructions(
    context: RunContextWrapper[UserContext], agent: Agent[UserContext]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."


agent = Agent[UserContext](
    name="Triage agent",
    instructions=dynamic_instructions,
)
```

### **Lifecycle events (hooks)**
Sometimes, you want to observe the lifecycle of an agent. For example, you may want to log events, or pre-fetch data when certain events occur. You can hook into the agent lifecycle with the hooks property. Subclass the `AgentHooks` class, and override the methods you're interested in.

### **Guardrails**
Guardrails allow you to run checks/validations on user input, in parallel to the agent running. For example, you could screen the user's input for relevance. Read more in the `guardrails` documentation.

## **Cloning/copying agents**
By using the `clone()` method on an agent, you can `duplicate` an `Agent`, and optionally change any properties you like.

```python
pirate_agent = Agent(
    name="Pirate",
    instructions="Write like a pirate",
    model="o3-mini",
)

robot_agent = pirate_agent.clone(
    name="Robot",
    instructions="Write like a robot",
)
```

### **Forcing tool use**
Supplying a list of tools doesn't always mean the LLM will use a tool. You can force tool use by setting ModelSettings.tool_choice. Valid values are:

- auto, which allows the LLM to decide whether or not to use a tool.
- required, which requires the LLM to use a tool (but it can intelligently decide which tool).
- none, which requires the LLM to not use a tool.
- Setting a specific string e.g. my_tool, which requires the LLM to use that specific tool.

## **`Note`**

To prevent infinite loops, the framework automatically resets `tool_choice` to "auto" after a tool call. This behavior is configurable via agent.reset_`tool_choice`. The infinite loop is because tool results are sent to the LLM, which then generates another tool call because of `tool_choice`, ad infinitum.

If you want the Agent to completely stop after a tool call (rather than continuing with auto mode), you can set `[Agent.tool_use_behavior="stop_on_first_tool"]` which will directly use the tool output as the final response without further LLM processing.